System Configuration Report



In [ ]:
import subprocess
import platform

def display_system_info():
    # Get CPU frequency
    try:
        cpu_freq = subprocess.getoutput("sysctl -n hw.cpufrequency")  # Returns in Hz
        cpu_freq_mhz = int(cpu_freq) / 1_000_000  # Convert to MHz
    except ValueError:
        cpu_freq_mhz = "Unknown"

    # Get memory size
    try:
        mem_bytes = subprocess.getoutput("sysctl -n hw.memsize")
        mem_gb = int(mem_bytes) / (1024 ** 3)  # Convert to GB
    except ValueError:
        mem_gb = "Unknown"

    # Get disk size
    try:
        disk_info = subprocess.getoutput("df -h / | tail -1 | awk '{print $2}'")
    except Exception as e:
        disk_info = "Unknown"

    # Get system information
    system = platform.system()
    node_name = platform.node()
    release = platform.release()
    version = platform.version()
    machine = platform.machine()
    cpu_name = platform.processor()
    physical_cores = subprocess.getoutput("sysctl -n hw.physicalcpu")
    logical_cores = subprocess.getoutput("sysctl -n hw.logicalcpu")

    # Display system information
    print("System Configuration Information:")
    print(f"Operating System: {system}")
    print(f"Node Name: {node_name}")
    print(f"System Version: {release}")
    print(f"Detailed System Version: {version}")
    print(f"CPU Architecture: {machine}")
    print(f"CPU Name: {cpu_name}")
    print(f"Physical Cores: {physical_cores}")
    print(f"Logical Cores: {logical_cores}")
    print(f"CPU Frequency: {cpu_freq_mhz} MHz" if cpu_freq_mhz != "Unknown" else "CPU Frequency: Unknown")
    print(f"Memory Size: {mem_gb:.2f} GB" if mem_gb != "Unknown" else "Memory Size: Unknown")
    print(f"Disk Size: {disk_info}" if disk_info != "Unknown" else "Disk Size: Unknown")

# Run the function
display_system_info()


In [ ]:
import subprocess
import re

def get_gpu_info():
    # Run system_profiler to get GPU information
    gpu_info_raw = subprocess.getoutput("system_profiler SPDisplaysDataType")
    
    # Extract GPU details
    gpu_info = {}
    for line in gpu_info_raw.splitlines():
        line = line.strip()
        if line.startswith("Chipset Model:"):
            gpu_info["Chipset Model"] = line.split(": ")[1]
        elif line.startswith("VRAM (Dynamic, Max):"):
            gpu_info["VRAM (Dynamic, Max)"] = line.split(": ")[1]
        elif line.startswith("VRAM (Total):"):
            gpu_info["VRAM (Total)"] = line.split(": ")[1]
        elif line.startswith("Vendor:"):
            gpu_info["Vendor"] = line.split(": ")[1]
        elif line.startswith("Metal Family:"):
            gpu_info["Metal Family"] = line.split(": ")[1]
        elif line.startswith("Displays:"):
            break  # End when display information starts
    
    return gpu_info

def format_gpu_info(gpu_info):
    report = "## GPU Information\n"
    for key, value in gpu_info.items():
        report += f"- **{key}**: {value}\n"
    return report

gpu_info = get_gpu_info()
markdown_report = format_gpu_info(gpu_info)
print(markdown_report)


In [7]:
import subprocess

def get_memory_info():
    # Get total memory size in bytes and convert to GB
    total_mem_bytes = int(subprocess.getoutput("sysctl -n hw.memsize"))
    total_mem_gb = total_mem_bytes / (1024 ** 3)

    # Get memory usage details from vm_stat
    vm_stat_output = subprocess.getoutput("vm_stat")
    pagesize = int(subprocess.getoutput("sysctl -n hw.pagesize"))

    # Extract page-based memory information and convert to GB
    memory_info = {
        "Total Memory": f"{total_mem_gb:.2f} GB"
    }
    for line in vm_stat_output.splitlines():
        if "Pages free:" in line:
            free_pages = int(line.split(":")[1].strip().replace(".", ""))
            memory_info["Free Memory"] = f"{(free_pages * pagesize) / (1024 ** 3):.2f} GB"
        elif "Pages active:" in line:
            active_pages = int(line.split(":")[1].strip().replace(".", ""))
            memory_info["Active Memory"] = f"{(active_pages * pagesize) / (1024 ** 3):.2f} GB"
        elif "Pages inactive:" in line:
            inactive_pages = int(line.split(":")[1].strip().replace(".", ""))
            memory_info["Inactive Memory"] = f"{(inactive_pages * pagesize) / (1024 ** 3):.2f} GB"
        elif "Pages wired down:" in line:
            wired_pages = int(line.split(":")[1].strip().replace(".", ""))
            memory_info["Wired Memory"] = f"{(wired_pages * pagesize) / (1024 ** 3):.2f} GB"
        elif "Pages purgeable:" in line:
            purgeable_pages = int(line.split(":")[1].strip().replace(".", ""))
            memory_info["Purgeable Memory"] = f"{(purgeable_pages * pagesize) / (1024 ** 3):.2f} GB"
    
    return memory_info

def format_memory_info(memory_info):
    report = "## Memory Information\n"
    for key, value in memory_info.items():
        report += f"- **{key}**: {value}\n"
    return report

memory_info = get_memory_info()
markdown_report = format_memory_info(memory_info)
print(markdown_report)


## Memory Information
- **Total Memory**: 16.00 GB
- **Free Memory**: 0.16 GB
- **Active Memory**: 3.21 GB
- **Inactive Memory**: 3.19 GB
- **Wired Memory**: 2.34 GB
- **Purgeable Memory**: 0.02 GB



In [9]:
import subprocess

def get_disk_space():
    # Run df command for root directory in human-readable format
    df_output = subprocess.getoutput("df -h /")
    
    # Print the output to debug
    print("Debug df output:\n", df_output)

    # Parse the output and extract disk space information
    lines = df_output.splitlines()
    header = lines[0].split()
    values = lines[1].split()

    # Check actual column names from the header and use them to create the dictionary
    disk_info = {
        header[0]: values[0],  # Filesystem
        header[1]: values[1],  # Size
        header[2]: values[2],  # Used
        header[3]: values[3],  # Avail
        header[4]: values[4],  # Capacity
        header[5]: values[5]   # Mounted on (may vary by system)
    }

    return disk_info

def format_disk_space_info(disk_info):
    report = "## Disk Space Information\n"
    report += f"- **Filesystem**: {disk_info.get('Filesystem', 'N/A')}\n"
    report += f"- **Total Size**: {disk_info.get('Size', 'N/A')}\n"
    report += f"- **Used**: {disk_info.get('Used', 'N/A')}\n"
    report += f"- **Available**: {disk_info.get('Avail', 'N/A')}\n"
    report += f"- **Usage Percentage**: {disk_info.get('Capacity', 'N/A')}\n"
    report += f"- **Mounted on**: {disk_info.get('Mounted', 'N/A')}\n"
    return report

# Execute the function
disk_info = get_disk_space()
markdown_report = format_disk_space_info(disk_info)
print(markdown_report)



Debug df output:
 Filesystem        Size    Used   Avail Capacity iused ifree %iused  Mounted on
/dev/disk3s1s1   460Gi    19Gi   259Gi     7%    393k  2.7G    0%   /
## Disk Space Information
- **Filesystem**: /dev/disk3s1s1
- **Total Size**: 460Gi
- **Used**: 19Gi
- **Available**: 259Gi
- **Usage Percentage**: 7%
- **Mounted on**: N/A

